# Using FuzzyWuzzy to match customers in the top global companies list
By Alex Infanzon

## Abstract

To ensure high data quality, data must be validated and cleansed. A common scenario for data scientists is, given two sets of similar data, normalize both data sets to have a common record for modelling. Fuzzy matching is a technique used in record linkage. It works with matches that may be less than 100% perfect. In this blog I am going to share a Jupyter notebook that compares and matches a two lists of customer names.

The code is written in Python 3.6 and leverages the fuzzy matching package (FuzzyWuzzy) to compare customer names. A match threshold confidence is set to match records from both data sources and create a summary of matching records.

## Introduction

To ensure high data quality, data warehouses or analytical warehouses must validate and cleanse incoming data from external sources. In many situations, clean tuples must match acceptable tuples in a reference data set. For example, customer name and description fields in a sales record from a customer list must match the pre-recorded name and description fields in another customer reference data set.

A significant challenge in such a scenario is to implement an efficient and accurate fuzzy match operation that can effectively clean an incoming tuple if it fails to match exactly with any tuple in the reference relation.

## Required libraries

For matching records you need to import fuzz and process from fuzzywuzzy. To load and manipulate matched records we will use a dataframe, so we need to import pandas. Optionally, if you want to add markdown cells from the python code you also need to import Markdown and display form the IPython.display library. 


In [1]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from IPython.display import Markdown, display
import pandas as pd

## Global Constants

This section has a definition of all the global constants used in the Python code. Change the file path to select different files. Also you can increase or decrease the matching to improve the matching results.

In [2]:
sourceFile = '/Users/ainfanzon/Public/trashme/forbes2000.csv'
#sourceFile = '/Users/ainfanzon/Public/trashme/fortune500'
targetFile = '/Users/ainfanzon/Public/trashme/orcl_cust.csv'
#targetFile = '/Users/ainfanzon/Public/trashme/all_cust.csv'#targetFile = '/Users/ainfanzon/Public/trashme/sap_cust.csv'
#targetFile = '/Users/ainfanzon/Public/trashme/msft_cust.csv'
#targetFile = '/Users/ainfanzon/Public/trashme/sas_cust.csv'

matches = 0     # count the number of matches
idx = 0         # index for the dataframe
threshold = 92  # matching threshold

## Matching function

The fuzzymatch function takes four parameters:

* s = Holds a list with company names from Salesforce, i.e., the source
* t = Holds the Top Global companies matching target
* d = Is the dataframe with the matching results
* i = Used for indexing rows in the dataframe

It evaluates if the matching score is greater than the threshold. If so, returns one to be added to the number of records matched, otherwise returns zero.

In [3]:
def fuzzymatch(s, t, d, i):
    new_name, score = process.extractOne(s, t)
    if score > threshold:  # threshold was defined in global constants
        d.loc[i] = [ s, new_name, score ]
        return 1
    return 0

## Main Section

The actions executed in this section are:

* Open source and target files
* Deduplicate the source records
* Match deduplicated records

### Open source and target files

The file names in the __*sourceFile*__ and __*targetFile*__ variables were defined in the global section above. The records are loaded to a list. 

In [4]:
source=[line.strip() for line in open(sourceFile)]
names=[line.strip() for line in open(targetFile)]


### Deduplicate the source records

We use the __*set*__ statement to deduplicate the Salesforce source records. Next the column names for the dataframe are defined.

In [5]:
lstNoDups = set(names)

df = pd.DataFrame(columns=('Salesforce source', 'Global Target', 'Score'))

### Match deduplicated records

Match deduplicated record using the __*fuzzymatch*__ function

In [6]:
for name in lstNoDups:
    matches = matches +  fuzzymatch(name, source, df, idx)
    idx = idx + 1

Below is a sample matching records and their matching score. To display all the record comment the line with the sample function and uncomment the other one.

In [7]:
display(df.sample(n=20))
#df.to_csv('/Users/ainfanzon/Public/trashme/SAP-Matching_Results.csv')

,Salesforce source,Global Target,Score
649,ConocoPhillips Norge,ConocoPhillips,95
782,Capital One Financial,Capital One Financial,100
580,CNO Financial,CNO Financial Group,95
809,Westar Energy,Westar Energy,100
707,"HENRY SCHEIN, INC.",Henry Schein,95
779,INTESA SANPAOLO SPA,Intesa Sanpaolo,95
355,Mitsubishi Electric,Mitsubishi Electric,100
137,Dassault Systemes EMEA,Dassault Systemes,95
345,Autoliv,Autoliv,100
554,XPO Logistics,XPO Logistics,100


## Findings

In this section the analysis results are presented.


In [8]:
conclusion_1 = "The total number of records extracted from Salesforce is " + str(len(names)) \
             + ". After de-duplication we have " + str(len(lstNoDups)) + " unique records." \
             + " From the list of unique records " + str(matches) + " matched to the top " \
             + str(len(source)) + " Global companies."
display(Markdown(conclusion_1))
conclusion_2 = "From Salesforce list of unique customers " + str(int(matches/len(source)*100)) \
             + "% are listed in the Global " + str(len(source)) + " companies. " \
             + "Which represent " + str(int(matches/len(lstNoDups)*100)) + "% of the installed base (# Matches/Salesforce deduped records)"
display(Markdown(conclusion_2))


The total number of records extracted from Salesforce is 1903. After de-duplication we have 822 unique records. From the list of unique records 35 matched to the top 2000 Global companies.

From Salesforce list of unique customers 1% are listed in the Global 2000 companies. Which represent 4% of the installed base (# Matches/Salesforce deduped records)